In [1]:
import sys, os
# if your notebook lives in notebooks/, and src/ is one level up:
sys.path.insert(0, os.path.abspath('..'))

# core modules
from src.map_viewer_v2 import MapViewer
from src.geocode_v2    import geocode_place
from src.area_calc_v2  import compute_area
from src.config_v2     import BHUVAN_LAYERS

# UI
import ipywidgets as widgets
from IPython.display import display, clear_output


In [2]:
# create the viewer & grab the leaflet Map
mv = MapViewer()
m  = mv.get_map()

# for printing area / errors
out = widgets.Output()


In [3]:
layer_selector = widgets.Dropdown(
    options=list(BHUVAN_LAYERS.keys()),
    value=list(BHUVAN_LAYERS.keys())[0],
    description='Basemap:'
)

def _on_layer_change(change):
    if change['name'] == 'value':
        mv.add_wms_layer(BHUVAN_LAYERS[change['new']])

layer_selector.observe(_on_layer_change, names='value')

display(layer_selector)


Dropdown(description='Basemap:', options=('Basemap', 'Satellite', 'Terrain (Elevation)', 'Geomorphology', 'LUL…

In [4]:
# Units dropdown
unit_selector = widgets.Dropdown(
    options=[('km²','km2'), ('ha','ha'), ('m²','m2')],
    value='km2',
    description='Units:'
)

# Place search
search_box = widgets.Text(description='Search:')
search_btn = widgets.Button(description='Go')

def on_search(b):
    with out:
        clear_output()
        try:
            lat, lon = geocode_place(search_box.value)
            m.center = (lat, lon)
            m.zoom   = 12
        except Exception as e:
            print(f"Geocoding error: {e}")

search_btn.on_click(on_search)

# layout and show
controls = widgets.HBox([search_box, search_btn, unit_selector])
display(controls)


In [5]:
def handle_draw(target, action, geo_json):
    # first ring of coords
    coords = geo_json['geometry']['coordinates'][0]
    area   = compute_area(coords, unit_selector.value)
    with out:
        clear_output()
        print(f"Area: {area:.3f} {unit_selector.label}")

mv.on_draw(handle_draw)


In [6]:
display(m)
display(out)


Map(center=[20.5937, 78.9629], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

Output()

In [7]:
import sys, os
# make sure src/ is on path
sys.path.insert(0, os.path.abspath('..'))

# core functionality
from src.map_viewer_v2 import MapViewer
from src.geocode_v2       import geocode_place
from src.area_calc_v2     import compute_area
from src.config_v2        import WMS_URL, WMS_VERSION, BHUVAN_LAYERS, DEFAULT_LAYER

# UI
import ipywidgets as widgets
from IPython.display   import display, clear_output


In [8]:
# 1) pick a default Bhuvan layer from the dict
default_layer = next(iter(BHUVAN_LAYERS.values()))

# 2) initialize the MapViewer with that layer
mv = MapViewer(layers=default_layer)
m  = mv.get_map()

# 3) widgets
search_box    = widgets.Text(description='Search:')
search_btn    = widgets.Button(description='Go')
unit_selector = widgets.Dropdown(
    options=[('km²','km2'), ('ha','ha'), ('m²','m2')],
    value='km2',
    description='Units:'
)
layer_selector = widgets.Dropdown(
    # build (label, value) pairs from your dict
    options=[(name, layer) for name, layer in BHUVAN_LAYERS.items()],
    value=default_layer,
    description='Basemap:'
)
out = widgets.Output()

# 4) pan/zoom on search
def on_search(b):
    with out:
        clear_output()
        try:
            lat, lon = geocode_place(search_box.value)
            m.center = (lat, lon)
            m.zoom   = 12
        except Exception as e:
            print("Geocoding error:", e)
search_btn.on_click(on_search)

# 5) swap WMS layer when dropdown changes
def on_layer_change(change):
    if change['name']=='value':
        mv.add_wms_layer(change['new'])
layer_selector.observe(on_layer_change, names='value')

# 6) area‐draw handler
def handle_draw(target, action, geo_json):
    coords = geo_json['geometry']['coordinates'][0]
    area   = compute_area(coords, unit_selector.value)
    with out:
        clear_output()
        print(f"Area: {area:.3f} {unit_selector.label}")

# hook the draw event on the MapViewer’s draw control
mv.on_draw(handle_draw)

# 7) layout everything
controls = widgets.HBox([
    search_box, search_btn,
    unit_selector, layer_selector
])
display(controls)
display(m)
display(out)


Map(center=[20.5937, 78.9629], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

Output()

In [ ]:
# ensure src/ is on the path
import sys, os
sys.path.insert(0, os.path.abspath('..'))

# core modules
from src.map_viewer_v2 import MapViewer
from src.geocode_v2    import geocode_place
from src.area_calc_v2  import compute_area
from src.config_v2     import BHUVAN_LAYERS, DEFAULT_LAYER

# UI
import ipywidgets as widgets
from IPython.display import display, clear_output


In [ ]:
# 1) Output area for messages
out = widgets.Output()

# 2) Unit selector
unit_selector = widgets.Dropdown(
    options=[('km²','km2'), ('ha','ha'), ('m²','m2')],
    value='km2',
    description='Units:'
)

# 3) Place search box + button
search_box = widgets.Text(description='Search:')
search_btn = widgets.Button(description='Go')

# 4) Layer selector
layer_selector = widgets.Dropdown(
    options=list(BHUVAN_LAYERS.keys()),
    value=DEFAULT_LAYER,
    description='Layer:'
)

# 5) Initialize MapViewer with the default Bhuvan layer
mv = MapViewer(layers=BHUVAN_LAYERS[DEFAULT_LAYER])
m  = mv.get_map()


In [ ]:
# Search handler
def on_search(b):
    with out:
        clear_output()
        try:
            lat, lon = geocode_place(search_box.value)
            m.center = (lat, lon)
            m.zoom   = 12
        except Exception as e:
            print(f"Geocoding error: {e}")

search_btn.on_click(on_search)

# Draw handler
def handle_draw(target, action, geo_json):
    # pull out the first ring
    coords = geo_json['geometry']['coordinates'][0]
    area   = compute_area(coords, unit_selector.value)
    with out:
        clear_output()
        print(f"Area: {area:.3f} {unit_selector.label}")

mv.draw_control.on_draw(handle_draw)

# Layer‐change handler
def on_layer_change(change):
    if change['type']=='change' and change['name']=='value':
        new_key = change['new']
        mv.add_wms_layer(BHUVAN_LAYERS[new_key])

layer_selector.observe(on_layer_change)


In [ ]:
controls = widgets.HBox([search_box, search_btn, unit_selector, layer_selector])
display(controls)

display(m)
display(out)

In [5]:
# Cell 1: imports & config
import sys, os
sys.path.insert(0, os.path.abspath('..'))  # so src/ is on your path

from src.map_viewer_v2 import MapViewer
from src.geocode_v2 import geocode_place
from src.area_calc_v2 import compute_area

# Bhuvan WMS endpoint & version
from src.config_v2 import WMS_URL, WMS_VERSION, DEFAULT_CENTER, DEFAULT_ZOOM

# Here’s our *complete* layer‐dictionary.
# Note: the raster basemaps have to be hand-typed—they aren’t in your txt file.
BHUVAN_LAYERS = {
    "OpenStreetMap":             "OpenStreetMap",
    "True-Color Satellite":      "bhuvan:TRUECOLOUR_3857",     # hand-picked WMS name
    "Terrain (Hillshade)":       "organization:Elevation",      # hand-picked WMS name

    # a few vector themes from your txt file:
    "Geomorphology (Maharashtra)": "geomorphology:MH_GM50K_0506",   # :contentReference[oaicite:0]{index=0}:contentReference[oaicite:1]{index=1}
    "LULC (Phase-2 UP)":            "sisdp_phase2:SISDP_P2_LULC_10K_2016_2019_UP_1",  # :contentReference[oaicite:2]{index=2}:contentReference[oaicite:3]{index=3}
    "Godavari Rivers":              "disaster:godavari_Rivers",     # :contentReference[oaicite:4]{index=4}:contentReference[oaicite:5]{index=5}
    "City Boundaries":              "owp:CITY_admin"                # :contentReference[oaicite:6]{index=6}:contentReference[oaicite:7]{index=7}
}

# UI
import ipywidgets as widgets
from IPython.display import display, clear_output


# Cell 2: build the MapViewer & the “layer” toggle
mv = MapViewer(center=DEFAULT_CENTER, zoom=DEFAULT_ZOOM, layers=BHUVAN_LAYERS["OpenStreetMap"])
m  = mv.get_map()

layer_selector = widgets.Dropdown(
    options=list(BHUVAN_LAYERS.keys()),
    value="OpenStreetMap",
    description="Map layer:"
)

def on_layer_change(change):
    if change["name"]=="value":
        mv.add_wms_layer(BHUVAN_LAYERS[change["new"]])
layer_selector.observe(on_layer_change, names="value")

display(layer_selector)
display(m)

# Cell 3: search box + units + draw control
out = widgets.Output()

# units dropdown
unit_selector = widgets.Dropdown(
    options=[('km²','km2'), ('ha','ha'), ('m²','m2')],
    value='km2',
    description='Units:'
)

# placename search
search_box = widgets.Text(description='Search:')
search_btn = widgets.Button(description='Go')
def on_search(b):
    with out:
        clear_output()
        try:
            lat, lon = geocode_place(search_box.value)
            m.center = (lat, lon); m.zoom = 12
        except Exception as e:
            print(f"Geocoding error: {e}")
search_btn.on_click(on_search)

controls = widgets.HBox([search_box, search_btn, unit_selector])
display(controls)

# draw handler
def handle_draw(target, action, geo_json):
    coords = geo_json['geometry']['coordinates'][0]
    area = compute_area(coords, unit_selector.value)
    with out:
        clear_output()
        print(f"Area: {area:.3f} {unit_selector.label}")

mv.on_draw(handle_draw)

display(out)


Dropdown(description='Map layer:', options=('OpenStreetMap', 'True-Color Satellite', 'Terrain (Hillshade)', 'G…

Map(center=[20.5937, 78.9629], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

Output()

In [4]:
import requests
import xml.etree.ElementTree as ET

from src.config_v2 import WMS_URL, WMS_VERSION

# fetch the capabilities XML
resp = requests.get(
    WMS_URL,
    params={
        "service": "WMS",
        "request": "GetCapabilities",
        "version": WMS_VERSION
    }
)
root = ET.fromstring(resp.content)

# find them all
layers = [n.text for n in root.findall(".//Layer/Name") if n.text]
print("Available Bhuvan WMS layers:\n")
print("\n".join(layers))


Available Bhuvan WMS layers:


